In [32]:
import numpy as np
import pandas as pd
import os

In [33]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import RandomizedSearchCV

In [34]:
os.getcwd()

'D:\\Desktop\\SUTD\\Term 8\\50.021 AI\\Final Project\\zhixuanRF'

In [35]:
df = pd.read_csv("numerical_data.csv")

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20112480 entries, 0 to 20112479
Data columns (total 14 columns):
 #   Column              Dtype  
---  ------              -----  
 0   #followers          int64  
 1   #friends            int64  
 2   #favorites          int64  
 3   Retweets            int64  
 4   yr_sin              float64
 5   yr_cos              float64
 6   time_sin            float64
 7   time_cos            float64
 8   nMentions           int64  
 9   nHashtags           int64  
 10  nURLs               int64  
 11  positive_sentiment  int64  
 12  negative_sentiment  int64  
 13  Retweeted           int64  
dtypes: float64(4), int64(10)
memory usage: 2.1 GB


In [37]:
retweeted = df[df["Retweeted"] > 0]

In [38]:
retweeted["log(Retweets)"] = np.log(retweeted["Retweets"])

<ipython-input-38-e12a81c7fea7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retweeted["log(Retweets)"] = np.log(retweeted["Retweets"])


In [39]:
y = retweeted["log(Retweets)"]
retweeted.drop(columns=["Retweeted", "Retweets", "log(Retweets)"], inplace=True)
X = retweeted

c:\python39\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [41]:
mms = MinMaxScaler()
mms.fit(X_train)
X_train = mms.transform(X_train)
X_test = mms.transform(X_test)

In [42]:
# rgr = RandomForestRegressor(max_depth=2, random_state=0)

Hyper-parameter tuning

In [43]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

print(rf_random.best_params_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


PicklingError: Could not pickle the task to send it to the workers.

In [10]:
# rgr.fit(X_train, y_train)

RandomForestRegressor(max_depth=2, random_state=0)

In [12]:

y_hat = rgr.predict(X_test)

In [14]:
# y_hat
y_hat

array([429417.27738715, 429417.27738715, 429417.27738715, ...,
       429417.27738715, 429417.27738715, 429417.27738715])

In [34]:
y_test_arr = y_test.values

In [35]:
msle = mean_squared_log_error(y_test_arr, y_hat)
print(mse)

11758965.136575485


In [27]:
from joblib import dump
dump(rgr, 'RFRegressor.joblib') 

In [15]:
from joblib import dump
dump(rgr, 'RFClassifier.joblib') 
# clf = load('filename.joblib') 

['RFClassifier.joblib']

Without performing log on the retweets: 3851185.024071577

After performing log on the retweets and regressing against log(Retweets): 1.184400578845887

In [47]:
from joblib import dump
dump(rgr, 'RFRegressor.joblib')

['RFRegressor.joblib']